# Pylops - numba

### Author: M.Ravasi

In this notebook I will investigate the benifit of adding numba to various operators

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import matplotlib.pyplot as plt
import scipy as sp

import pylops

from scipy.sparse import csr_matrix, vstack, rand
from scipy.linalg import lstsq, solve, pinv
from scipy.sparse.linalg import LinearOperator as spLinearOperator
from scipy.sparse.linalg import LinearOperator, cg, lsqr

from pylops.utils                      import dottest
from pylops.utils.wavelets             import *
from pylops.utils.seismicevents        import *
from pylops.basicoperators             import *
from pylops.signalprocessing             import *
from pylops.waveeqprocessing.mdd       import *
from pylops.optimization.leastsquares  import *
from pylops.optimization.sparsity  import IRLS as IRLSpylops
from pylops.optimization.sparsity  import FISTA

from numba import jit

## Slant stack

In [12]:
par = {'nt': 11, 'nhx': 21, 'ny': 10, 'npx':31, 'pxmax':1e-2,
       'centeredh': True, 'kind': 'linear',
       'interp':True, 'onthefly':True} # linear, centered, linear interp

dt, dh = 0.005, 1
t = np.arange(par['nt']) * dt
h = np.arange(par['nhx']) * dh
px = np.linspace(0, par['pxmax'], par['npx'])
x = np.zeros((par['npx'], par['nt']))
x[2, par['nt']//2] = 1
Rop = Radon2D(t, h, px, centeredh=par['centeredh'],
              interp=False, kind=par['kind'],
              dtype='float64')
dottest(Rop, par['nhx']*par['nt'], par['npx']*par['nt'],
        complexflag=0)
 
y = Rop * x.flatten()
xadj = Rop.H * y.flatten()

%timeit -n 10 Rop * x.flatten()
%timeit -n 10 Rop.H * y.flatten()

7.54 ms ± 660 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
9.91 ms ± 1.28 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [13]:
Rop = Radon2D(t, h, px, centeredh=par['centeredh'],
              interp=False, kind=par['kind'], engine='numba',
              dtype='float64')
dottest(Rop, par['nhx']*par['nt'], par['npx']*par['nt'],
        complexflag=0)

y1 = Rop * x.flatten()
xadj1 = Rop.H * y1.flatten()

%timeit -n 10 Rop * x.flatten()
%timeit -n 10 Rop.H * y.flatten()

70.5 µs ± 12.2 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
101 µs ± 24.3 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
